In [16]:
import sqlite3
import pandas as pd
import numpy as np

In [3]:
conn = sqlite3.connect("bluebikes.db")

In [4]:
stations = pd.read_sql_query("SELECT * FROM stations", conn)
rides = pd.read_sql_query("SELECT * FROM rides", conn)


In [5]:
stations.head()

,id,station_name
0,None,None
1,None,87-101 Cambridgepark Drive
2,None,Boylston St at Berkeley St
3,None,Boylston St at Exeter St
4,None,Boylston St at Fairfield St


In [6]:
rides.head()

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual
0,0CB95C5072A9A0E5,classic_bike,2025-04-13 21:06:54.496,2025-04-13 21:17:17.456,B32027,E32003,member
1,5C425780D5F59109,classic_bike,2025-04-28 12:38:52.585,2025-04-28 12:41:22.185,A32044,A32049,member
2,76808A97675DCECE,classic_bike,2025-04-14 11:36:20.428,2025-04-14 11:58:38.629,S32003,C32004,member
3,00E6835C21328359,classic_bike,2025-04-14 18:06:25.319,2025-04-14 18:21:04.129,M32053,B32058,member
4,484F5E541E4A4AD7,classic_bike,2025-04-24 18:32:17.408,2025-04-24 18:52:44.968,M32053,B32058,member


In [14]:
rides["ended_at_ts"] = pd.to_datetime(rides["ended_at"])
rides["started_at_ts"] = pd.to_datetime(rides["started_at"])

rides["length"] = rides["ended_at_ts"] - rides["started_at_ts"]
rides["length"] = rides["length"]/pd.Timedelta(1,'minute')
rides.head()

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual,ended_at_ts,started_at_ts,length
0,0CB95C5072A9A0E5,classic_bike,2025-04-13 21:06:54.496,2025-04-13 21:17:17.456,B32027,E32003,member,2025-04-13 21:17:17.456,2025-04-13 21:06:54.496,10.382667
1,5C425780D5F59109,classic_bike,2025-04-28 12:38:52.585,2025-04-28 12:41:22.185,A32044,A32049,member,2025-04-28 12:41:22.185,2025-04-28 12:38:52.585,2.493333
2,76808A97675DCECE,classic_bike,2025-04-14 11:36:20.428,2025-04-14 11:58:38.629,S32003,C32004,member,2025-04-14 11:58:38.629,2025-04-14 11:36:20.428,22.303350
3,00E6835C21328359,classic_bike,2025-04-14 18:06:25.319,2025-04-14 18:21:04.129,M32053,B32058,member,2025-04-14 18:21:04.129,2025-04-14 18:06:25.319,14.646833
4,484F5E541E4A4AD7,classic_bike,2025-04-24 18:32:17.408,2025-04-24 18:52:44.968,M32053,B32058,member,2025-04-24 18:52:44.968,2025-04-24 18:32:17.408,20.459333


In [48]:
same_station_rides = rides[rides["start_station_id"]==rides["end_station_id"]]
avg_same_station_length = np.average(same_station_rides["length"])
avg_length = np.average(rides["length"])
print(f"The average length of a ride that starts and stops at the same station is {round(avg_same_station_length,2)} minutes") # almost 34 minutes
print(f"Rides that start and stop at the location make up {round(len(same_station_rides)/len(rides) * 100.0,2)} percent of rides") ## 3.2 percent

The average length of a ride that starts and stops at the same station is 33.97 minutes
Rides that start and stop at the location make up 3.23 percent of rides


In [ ]:
member_rides = rides[rides["member_casual"]=="member"]
casual_rides = rides[rides["member_casual"]=="casual"]
print(f"The average member ride length is {round(np.average(member_rides["length"]),2)} minutes")
print(f"The average casual rider ride length is {round(np.average(casual_rides["length"]),2)} minutes")
print(f"Members take an average of {round(len(member_rides)/len(casual_rides),2)} more rides than casual riders")

The average member ride length is: 12.54 minutes
The average casual rider ride length is:25.64 minutes
Members take an average of 2.66 more rides than casual riders


In [ ]:
rides["hour"] = rides["started_at_ts"].dt.hour

def time_of_day(hour):
    if hour < 7:
        return "early morning"
    elif 7<=hour<12:
        return "morning"
    elif 12<=hour<17:
        return "afternoon"
    elif 17<=hour<20:
        return "evening"
    else:
        return "night"
    
rides["time_of_day"] = rides["hour"].apply(time_of_day)
rides[["started_at","time_of_day"]]

for time_of_day in ["early morning","morning","afternoon","evening","night"]:


SyntaxError: expected ':' (2772211012.py, line 18)